전처리

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -qq "/content/drive/MyDrive/메타버스아카데미/프로젝트/EPL선수몸값예측프로젝트/0616/Soccer_Player_data.zip"

In [4]:
import pandas as pd
import numpy as np
#데이터 불러오기
#포지션 공격수,수비수,미드필더,골키퍼
position = ['Forward','Midfielder','Defender','Goalkeeper']
features = ['Goals', 'Assists', 'Appearances', 'Minutes Played', 'Yellow Cards', 'Red Cards', 'Substituted On', 'Substituted Off', 'Shots', 'Shots On Target', 'Hit Woodwork', 'Goals From Header', 'Goals From Penalty', 'Goals From Freekick', 'Offsides', 'Touches', 'Passes', 'Through Balls', 'Crosses', 'Corners Taken', 'Interceptions', 'Blocks', 'Tackles', 'Last Man Tackles', 'Clearances', 'Headed Clearances', 'Aerial Battles Won', 'Own Goals', 'Errors Leading To Goal', 'Penalties Conceded', 'Aerial Battles Lost', 'Clean Sheets', 'Goals Conceded', 'Saves', 'Penalties Saved', 'Punches', 'High Claims', 'Sweeper Clearances', 'Throw Outs', 'Goal Kicks']

#각 데이터를 불러모아 그 csv파일의 이름으로 데이터프레임으로 만들어 주었습니다.
for P in position:
  for F in features:
    #이때 변수명에 space가 들어가는걸 방지하기 위해
    if len(F.split()) != 1 : #공백으로 나눠진다면
      newF = '_'.join(F.split()) #공백이 아닌 언더바로 바꾸기
      globals()['{}_{}'.format(P,newF)] = pd.read_csv('{}_{}.csv'.format(P,F),encoding='utf-8-sig')
      #컬럼명도 공백이 아닌 언더바로 바꾸어줍니다.
      globals()['{}_{}'.format(P,newF)].rename(columns={F:newF},inplace=True)
    else:
      globals()['{}_{}'.format(P,F)] = pd.read_csv('{}_{}.csv'.format(P,F),encoding='utf-8-sig')

In [5]:
#앞으로 사용할 features 도 공백이 아닌 언더바로 바꾸어 줍니다.
for F in features:
  if len(F.split()) != 1 :
    newF = '_'.join(F.split())
    features[features.index(F)] = newF

In [6]:
#모두 그이름으로 데이터 프레임이 된 모습을 확인할 수 있고, ',' 때문에 문자인 데이터들을 숫자로 변환해줍니다.
for P in position:
  for F in features:
    if globals()['{}_{}'.format(P,F)][F].dtype == object:
      globals()['{}_{}'.format(P,F)][F] = globals()['{}_{}'.format(P,F)][F].str.replace(pat=r'[^\w]', repl=r'', regex=True) #특수기호 지우기
      globals()['{}_{}'.format(P,F)][F] = pd.to_numeric(globals()['{}_{}'.format(P,F)][F],errors='ignore') #지운후 숫자로 바꿀수있으면 바꾸기
    globals()['{}_{}'.format(P,F)].info()
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Player       93 non-null     object
 1   Club         93 non-null     object
 2   Nationality  93 non-null     object
 3   Goals        93 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 3.0+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Player       79 non-null     object
 1   Club         79 non-null     object
 2   Nationality  79 non-null     object
 3   Assists      79 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 2.6+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124 entries, 0 to 123
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Player     

In [7]:
#모든 데이터들 Player 기준으로 통합
for P in position:
  #일단 Position으로된 이름의 데이터프레임을 featuers 1,2번만 따서 만들고
  globals()['{}'.format(P)] = pd.merge(globals()['{}_{}'.format(P,features[0])],globals()['{}_{}'.format(P,features[1])], how='outer', on=['Player','Club','Nationality'])
  for F in features[2:] :
    #그뒤로는 쭉 붙입니다.
    globals()['{}'.format(P)] = pd.merge(globals()['{}'.format(P)],globals()['{}_{}'.format(P,F)], how='outer', on=['Player','Club','Nationality'])

In [8]:
#Merge함수를 쓸때 how 파라미터를 outer로 했기 때문에 없는데이터는 결측치로 됩니다. 결측치 처리
#그리고 결측치라는것은 없다는것 이기 때문에 0으로 대체해 줍니다. ex) Goal = Nan 은 골을 넣지않아 데이터가 없다는 의미
Goalkeeper.fillna(0, inplace=True)
Forward.fillna(0, inplace=True)
Midfielder.fillna(0, inplace=True)
Defender.fillna(0, inplace=True)

In [9]:
#포지션별로 모든 데이터가 들어간 데이터 프레임 완성
Goalkeeper.info()
Forward.info()
Midfielder.info()
Defender.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 43 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Goals                   39 non-null     float64
 1   Player                  39 non-null     object 
 2   Club                    39 non-null     object 
 3   Nationality             39 non-null     object 
 4   Assists                 39 non-null     float64
 5   Appearances             39 non-null     int64  
 6   Minutes_Played          39 non-null     int64  
 7   Yellow_Cards            39 non-null     float64
 8   Red_Cards               39 non-null     float64
 9   Substituted_On          39 non-null     float64
 10  Substituted_Off         39 non-null     float64
 11  Shots                   39 non-null     float64
 12  Shots_On_Target         39 non-null     float64
 13  Hit_Woodwork            39 non-null     float64
 14  Goals_From_Header       39 non-null     floa

In [12]:
#이제 연봉데이터를 불러옵니다.
salary = pd.read_csv('/content/drive/MyDrive/메타버스아카데미/프로젝트/EPL선수몸값예측프로젝트/0616/salary.csv',encoding='utf-8-sig')
salary

,Player,Value
0,Erling Haaland,170000000.0
1,Bukayo Saka,110000000.0
2,Phil Foden,110000000.0
3,Harry Kane,90000000.0
4,Enzo Fernández,85000000.0
...,...,...
586,Andy Lonergan,200000.0
587,Tom McGill,200000.0
588,Will Dennis,200000.0
589,Willy Caballero,200000.0


In [13]:
#결측치 확인
salary.isnull().sum()

Player    0
Value     0
dtype: int64

In [14]:
#포텐셜이 결측치인 경우는 그 주변인데 측정하기 어려워서 이므로, Rating값을 사용해준다.
salary['Potential'] = np.where(pd.notnull(salary['Potential']) == True, salary['Potential'], salary['Rating'])
salary.isnull().sum()

KeyError: 'Potential'

In [15]:
#wage 값 숫자형으로 바꿔주기
temp = []
for i in salary['Wage']:
  #먼저 앞에 두자리를 자르고, ','문자로 분리한 다음 합쳐준것을 int형으로 바꿔줍니다. 이때 ','문자가없더라도 크기가 하나인 str로 나눠지기 때문에 영향이 없습니다.
  temp.append(int(''.join(i[2:].split((',')))))
salary['Wage'] = temp
salary.info() #int형으로 바뀐모습

KeyError: 'Wage'

In [ ]:
#sell_value 값 숫자형으로 바꿔주기
temp = []
for i in salary['Sell_value']:
  #먼저 앞에 두자리를 자르고, ','문자로 분리한 다음 합쳐준것을 int형으로 바꿔줍니다. 이때 ','문자가없더라도 크기가 하나인 str로 나눠지기 때문에 영향이 없습니다.
  try:
    temp.append(int(''.join(i[2:].split((',')))))
  except:
    temp.append(temp[-1])
salary['Sell_value'] = temp
salary.info() #int형으로 바뀐모습

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Player      699 non-null    object 
 1   Potential   699 non-null    float64
 2   Rating      699 non-null    int64  
 3   Wage        699 non-null    int64  
 4   Sell_value  699 non-null    int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 27.4+ KB


In [ ]:
#지금까지 정한 선수들의 데이터와 연봉데이터를 합칩니다. 다만 이땐 연봉데이터가 없는 선수는 데이터에 활용할 수 없으므로, how 파라미터를 inner을 사용합니다.
#데이터 전처리 완료
Goalkeeper  = pd.merge(Goalkeeper,salary,how = 'inner',on = 'Player')
Forward  = pd.merge(Forward,salary,how = 'inner',on = 'Player')
Midfielder  = pd.merge(Midfielder,salary,how = 'inner',on = 'Player')
Defender  = pd.merge(Defender,salary,how = 'inner',on = 'Player')

In [ ]:
Goalkeeper.to_csv('Goalkeeper_20.csv',encoding ='utf-8-sig')
Forward.to_csv('Forward_20.csv',encoding ='utf-8-sig')
Midfielder.to_csv('Midfielder_20.csv',encoding ='utf-8-sig')
Defender.to_csv('Defender_20.csv',encoding ='utf-8-sig')

In [ ]:
Forward

,Unnamed: 0,Player,Goals,Assists,Appearances,Minutes_Played,Yellow_Cards,Red_Cards,Substituted_On,Substituted_Off,Shots,Shots_On_Target,Hit_Woodwork,Goals_From_Header,Goals_From_Penalty,Goals_From_Freekick,Offsides,Touches,Passes,Through_Balls,Crosses,Corners_Taken,Interceptions,Blocks,Tackles,Last_Man_Tackles,Clearances,Headed_Clearances,Aerial_Battles_Won,Own_Goals,Errors_Leading_To_Goal,Penalties_Conceded,Aerial_Battles_Lost,Clean_Sheets,Goals_Conceded,Potential,Rating,Wage,Club_-,Club_Arsenal,Club_Aston Villa,Club_Bournemouth,Club_Brentford,Club_Brighton & Hove Albion,Club_Chelsea,Club_Crystal Palace,Club_Everton,Club_Fulham,Club_Leeds United,Club_Leicester City,Club_Liverpool,Club_Manchester City,Club_Manchester United,Club_Newcastle United,Club_Norwich City,Club_Nottingham Forest,Club_Sheffield United,Club_Southampton,Club_Tottenham Hotspur,Club_Watford,Club_West Bromwich Albion,Club_West Ham United,Club_Wolverhampton Wanderers,Nationality_Albania,Nationality_Algeria,Nationality_Angola,Nationality_Argentina,Nationality_Belgium,Nationality_Brazil,Nationality_Burkina Faso,Nationality_Colombia,Nationality_Cote D’Ivoire,Nationality_Ecuador,Nationality_Egypt,Nationality_England,Nationality_Finland,Nationality_France,Nationality_Gabon,Nationality_Germany,Nationality_Ghana,Nationality_Greece,Nationality_Iran,Nationality_Ireland,Nationality_Israel,Nationality_Italy,Nationality_Jamaica,Nationality_Japan,Nationality_Mauritania,Nationality_Mexico,Nationality_Netherlands,Nationality_New Zealand,Nationality_Nigeria,Nationality_Norway,Nationality_Paraguay,Nationality_Portugal,Nationality_Scotland,Nationality_Senegal,Nationality_Serbia,Nationality_South Korea,Nationality_Spain,Nationality_Sweden,Nationality_Switzerland,Nationality_United States,Nationality_Uruguay,Nationality_Venezuela,Nationality_Wales,Nationality_Zambia,Nationality_[object Object],Value
0,0,Harry Kane,23.00,14.00,35.00,3085.00,1.00,0.00,0.00,5.00,137.00,53.00,9.00,4.00,4.00,0.00,13.00,1561.00,908.00,6.00,38.00,0.00,11.00,11.00,22.00,0.00,39.00,35.00,76.00,0.00,0.00,0.00,114.00,11.00,39.00,93.00,91,220000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,87000000.00
1,1,Mohamed Salah,22.00,5.00,37.00,3079.00,0.00,0.00,3.00,7.00,126.00,52.00,1.00,2.00,6.00,0.00,13.00,1842.00,1263.00,10.00,29.00,0.00,1.00,0.00,18.00,0.00,10.00,5.00,18.00,0.00,1.00,0.00,42.00,8.00,41.00,92.00,90,220000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84000000.00
2,2,Mohamed Salah,22.00,5.00,37.00,3079.00,0.00,0.00,3.00,7.00,126.00,52.00,1.00,2.00,6.00,0.00,13.00,1842.00,1263.00,10.00,29.00,0.00,1.00,0.00,18.00,0.00,10.00,5.00,18.00,0.00,1.00,0.00,42.00,8.00,41.00,92.00,90,220000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84000000.00
3,3,Patrick Bamford,17.00,7.00,38.00,3062.00,3.00,0.00,1.00,13.00,107.00,48.00,4.00,2.00,2.00,0.00,18.00,920.00,509.00,2.00,3.00,2.00,10.00,4.00,21.00,0.00,37.00,29.00,47.00,0.00,0.00,0.00,74.00,9.00,50.00,70.00,65,38500,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11250000.00
4,4,Dominic Calvert-Lewin,16.00,0.00,33.00,2874.00,3.00,0.00,1.00,6.00,83.00,46.00,1.00,7.00,0.00,0.00,18.00,1144.00,675.00,1.00,9.00,1.00,6.00,6.00,17.00,0.00,33.00,31.00,154.00,0.00,0.00,0.00,135.00,8.00,39.00,82.00,76,77000,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39500000.00
5,5,Jamie Vardy,15.00,9.00,34.00,2844.00,1.00,0.00,3.00,7.00,82.00,38.00,4.00,1.00,8.00,0.00,36.00,778.00,434.00,0.00,20.00,0.00,4.00,5.00,18.00,0.00,31.00,18.00,50.00,0.00,0.00,0.00,70.00,4.00,44.00,82.00,80,175000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0

3년치 데이터 병합하기

In [ ]:
import pandas as pd
import numpy as np
#데이터 불러오기
#포지션 공격수,수비수,미드필더,골키퍼
position = ['Forward','Midfielder','Defender','Goalkeeper']
features = ['Goals', 'Assists', 'Appearances', 'Minutes Played', 'Yellow Cards', 'Red Cards', 'Substituted On', 'Substituted Off', 'Shots', 'Shots On Target', 'Hit Woodwork', 'Goals From Header', 'Goals From Penalty', 'Goals From Freekick', 'Offsides', 'Touches', 'Passes', 'Through Balls', 'Crosses', 'Corners Taken', 'Interceptions', 'Blocks', 'Tackles', 'Last Man Tackles', 'Clearances', 'Headed Clearances', 'Aerial Battles Won', 'Own Goals', 'Errors Leading To Goal', 'Penalties Conceded', 'Aerial Battles Lost', 'Clean Sheets', 'Goals Conceded', 'Saves', 'Penalties Saved', 'Punches', 'High Claims', 'Sweeper Clearances', 'Throw Outs', 'Goal Kicks']

#3년치 데이터 불러오기 그 csv파일의 이름으로 데이터프레임으로 만들어 주었습니다.
for P in position:
  for Y in ['20','21','22']:
      globals()['{}_{}'.format(P,Y)] = pd.read_csv('{}_{}.csv'.format(P,Y),encoding='utf-8-sig')

In [ ]:
for P in position:
  globals()['{}_22'.format(P)].rename(columns={'Value':'Sell_value'},inplace = True)

In [ ]:
Defender_22.head()

,Unnamed: 0,Player,Club,Nationality,Goals,Assists,Appearances,Minutes_Played,Yellow_Cards,Red_Cards,Substituted_On,Substituted_Off,Shots,Shots_On_Target,Hit_Woodwork,Goals_From_Header,Goals_From_Penalty,Goals_From_Freekick,Offsides,Touches,Passes,Through_Balls,Crosses,Corners_Taken,Interceptions,Blocks,Tackles,Last_Man_Tackles,Clearances,Headed_Clearances,Aerial_Battles_Won,Own_Goals,Errors_Leading_To_Goal,Penalties_Conceded,Aerial_Battles_Lost,Clean_Sheets,Goals_Conceded,Saves,Penalties_Saved,Punches,High_Claims,Sweeper_Clearances,Throw_Outs,Goal_Kicks,Potential,Rating,Wage,Sell_value
0,0,Rasmus Kristensen,Leeds United,Denmark,3.00,1.00,26,1961,4.00,0.00,5.00,4.00,13.00,5.00,0.00,0.00,0.00,0.00,1.00,1309,833,0.00,35.00,0.00,35.00,14.00,62.00,3.00,55.00,21.00,34.00,0.00,0.00,0.00,27.00,1.00,53.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,72.00,66,52200,15000000
1,1,Ben Mee,Brentford,England,3.00,2.00,37,3272,2.00,0.00,0.00,3.00,28.00,7.00,1.00,2.00,0.00,0.00,1.00,2163,1658,0.00,7.00,0.00,48.00,42.00,45.00,1.00,116.00,64.00,132.00,0.00,0.00,0.00,98.00,12.00,40.00,0.00,0.00,0.00,0.00,0.00,0.00,2.00,72.00,68,58000,1800000
2,2,Ethan Pinnock,-,Jamaica,3.00,0.00,30,2700,0.00,0.00,0.00,0.00,15.00,9.00,0.00,0.00,0.00,0.00,0.00,1580,1092,0.00,10.00,0.00,35.00,31.00,52.00,2.00,214.00,131.00,105.00,1.00,2.00,0.00,54.00,10.00,34.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,69.00,68,34800,12000000
3,3,Pedro Porro,-,Spain,3.00,3.00,15,1134,0.00,0.00,2.00,7.00,26.00,7.00,1.00,0.00,0.00,0.00,1.00,863,478,7.00,77.00,24.00,15.00,2.00,25.00,0.00,20.00,6.00,1.00,0.00,1.00,0.00,11.00,1.00,26.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,82.00,75,31500,35000000
4,4,Virgil van Dijk,Liverpool,Netherlands,3.00,1.00,32,2835,3.00,0.00,0.00,1.00,30.00,10.00,0.00,2.00,0.00,0.00,3.00,2893,2581,1.00,1.00,0.00,31.00,16.00,27.00,0.00,135.00,61.00,98.00,0.00,1.00,1.00,35.00,11.00,36.00,0.00,0.00,0.00,0.00,0.00,0.00,17.00,92.00,89,278400,45000000


In [ ]:
for P in position:
  globals()['{}'.format(P)] = pd.concat([globals()['{}_{}'.format(P,'20')],globals()['{}_{}'.format(P,'21')]])
  #globals()['{}'.format(P)] = pd.concat([globals()['{}'.format(P)],globals()['{}_{}'.format(P,'22')]])

In [ ]:
#중복 행 제거
for P in position:
  globals()['{}'.format(P)].drop_duplicates()

In [ ]:
#앞으로 사용할 features 도 공백이 아닌 언더바로 바꾸어 줍니다.
for F in features:
  if len(F.split()) != 1 :
    newF = '_'.join(F.split())
    features[features.index(F)] = newF

In [ ]:
# 특정 열의 모든 값이 0인 열 제거

for P in position:
  for F in features:
    if all(globals()['{}'.format(P)][F] == 0):
      globals()['{}'.format(P)] = globals()['{}'.format(P)].drop(F, axis=1)

In [ ]:
#뛰고있는 클럽명과 나라이름 원핫 인코딩 하기
for P in position:
  globals()['{}'.format(P)] = pd.get_dummies(globals()['{}'.format(P)],columns  = ['Club','Nationality'])

In [ ]:
#sell_value를 끝 열로 옮기기
for P in position:
  globals()['{}'.format(P)]['Value'] = globals()['{}'.format(P)]['Sell_value']
  globals()['{}'.format(P)].drop(['Sell_value'], axis=1,inplace = True)

In [ ]:
for P in position:
  print(globals()['{}'.format(P)]['Value'])

0       87000000
1       84000000
2       84000000
3       11250000
4       39500000
5       19500000
6       15500000
7       47000000
8       39500000
9       34000000
10      46500000
11      84000000
12      57000000
13      57000000
14      13250000
15      55000000
16       8500000
17      14500000
18      10000000
19      38000000
20      79000000
21      77000000
22      55000000
23      47000000
24      35000000
25      21000000
26      21000000
27      21000000
28       7500000
29      38000000
30      47000000
31      33000000
32      38000000
33      27000000
34      16000000
35      55000000
36      11250000
37      10750000
38      28500000
39      30000000
40      62000000
41      14250000
42       7750000
43      41500000
44       5750000
45      55000000
46      55000000
47       2500000
48      18250000
49      18250000
50      28500000
51      22500000
52      19750000
53      37000000
54      32000000
55       1700000
56      18500000
57      18500000
58      125000

In [ ]:
Goalkeeper.head()

,Unnamed: 0,Player,Goals,Assists,Appearances,Minutes_Played,Yellow_Cards,Red_Cards,Substituted_On,Substituted_Off,Shots,Shots_On_Target,Goals_From_Header,Touches,Passes,Through_Balls,Interceptions,Tackles,Last_Man_Tackles,Clearances,Headed_Clearances,Aerial_Battles_Won,Own_Goals,Errors_Leading_To_Goal,Penalties_Conceded,Aerial_Battles_Lost,Clean_Sheets,Goals_Conceded,Saves,Penalties_Saved,Punches,High_Claims,Sweeper_Clearances,Throw_Outs,Goal_Kicks,Potential,Rating,Wage,Club_-,Club_Arsenal,Club_Aston Villa,Club_Bournemouth,Club_Brentford,Club_Brighton & Hove Albion,Club_Chelsea,Club_Crystal Palace,Club_Everton,Club_Fulham,Club_Leeds United,Club_Leicester City,Club_Liverpool,Club_Manchester City,Club_Newcastle United,Club_Norwich City,Club_Nottingham Forest,Club_Southampton,Club_Tottenham Hotspur,Club_Watford,Club_West Bromwich Albion,Club_West Ham United,Club_Wolverhampton Wanderers,Nationality_Argentina,Nationality_Australia,Nationality_Austria,Nationality_Bosnia & Herzegovina,Nationality_Brazil,Nationality_Costa Rica,Nationality_Denmark,Nationality_England,Nationality_France,Nationality_Germany,Nationality_Ireland,Nationality_Netherlands,Nationality_Northern Ireland,Nationality_Norway,Nationality_Poland,Nationality_Portugal,Nationality_Scotland,Nationality_Slovakia,Nationality_Spain,Nationality_Sweden,Nationality_United States,Nationality_Wales,Value
0,0,Alisson,1.00,0.00,33,2970,1.00,0.00,0.00,0.00,1.00,1.00,1.00,1321,988,0.00,2.00,3.00,1.00,43.00,4.00,14.00,0.00,3.00,1.00,1.00,10.00,32.00,84.00,1.00,17.00,20.00,25.00,165.00,173,90.00,87,130000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59000000
1,1,Sam Johnstone,0.00,1.00,37,3330,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1531,1108,0.00,2.00,0.00,0.00,38.00,3.00,7.00,0.00,0.00,1.00,1.00,6.00,74.00,166.00,1.00,23.00,20.00,14.00,182.00,296,70.00,64,33000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8000000
2,2,Sam Johnstone,0.00,1.00,37,3330,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1531,1108,0.00,2.00,0.00,0.00,38.00,3.00,7.00,0.00,0.00,1.00,1.00,6.00,74.00,166.00,1.00,23.00,20.00,14.00,182.00,296,70.00,64,33000,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8000000
3,3,Ederson,0.00,1.00,36,3240,3.00,0.00,0.00,0.00,0.00,0.00,0.00,1237,959,0.00,0.00,3.00,2.00,17.00,1.00,7.00,0.00,1.00,0.00,1.00,19.00,28.00,66.00,1.00,6.00,19.00,16.00,161.00,153,90.00,84,110000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55000000
4,4,Ederson,0.00,1.00,36,3240,3.00,0.00,0.00,0.00,0.00,0.00,0.00,1237,959,0.00,0.00,3.00,2.00,17.00,1.00,7.00,0.00,1.00,0.00,1.00,19.00,28.00,66.00,1.00,6.00,19.00,16.00,161.00,153,90.00,84,110000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55000000


In [ ]:
Goalkeeper.iloc[:,1:].to_csv('Goalkeeper.csv',encoding ='utf-8-sig')
Forward.iloc[:,1:].to_csv('Forward.csv',encoding ='utf-8-sig')
Midfielder.iloc[:,1:].to_csv('Midfielder.csv',encoding ='utf-8-sig')
Defender.iloc[:,1:].to_csv('Defender.csv',encoding ='utf-8-sig')

연봉자료 전처리


In [ ]:
import pandas as pd
import numpy as np
position = ['Forward','Midfielder','Defender','Goalkeeper']
year = ['20','21','22']

#이제 연봉데이터를 불러옵니다.
for P in position:
  for Y in year:
    globals()['{}_wage_{}'.format(P,Y)] = pd.read_csv('{}_wage_{}.csv'.format(P,Y),encoding='utf-8-sig')


In [ ]:
#포텐셜이 결측치인 경우는 그 주변인데 측정하기 어려워서 이므로, Rating값을 사용해준다.
for P in position:
  for Y in year:
    globals()['{}_wage_{}'.format(P,Y)]['Potential'] = np.where(pd.notnull(globals()['{}_wage_{}'.format(P,Y)]['Potential']) == True, globals()['{}_wage_{}'.format(P,Y)]['Potential'],globals()['{}_wage_{}'.format(P,Y)]['Rating'])
    #globals()['{}_wage_{}'.format(P,Y)].isnull().sum()

In [ ]:
#wage 값 숫자형으로 바꿔주기
for P in position:
  for Y in year:
    temp = []
    for i in globals()['{}_wage_{}'.format(P,Y)]['Wage']:
      #먼저 앞에 두자리를 자르고, ','문자로 분리한 다음 합쳐준것을 int형으로 바꿔줍니다. 이때 ','문자가없더라도 크기가 하나인 str로 나눠지기 때문에 영향이 없습니다.
      temp.append(int(''.join(i[2:].split((',')))))
    globals()['{}_wage_{}'.format(P,Y)]['Wage'] = temp
    globals()['{}_wage_{}'.format(P,Y)].info() #int형으로 바뀐모습

In [ ]:
#sell_value 값 숫자형으로 바꿔주기
for P in position:
  for Y in year:
    temp = []
    for i in globals()['{}_wage_{}'.format(P,Y)]['Sell_value']:
      #먼저 앞에 두자리를 자르고, ','문자로 분리한 다음 합쳐준것을 int형으로 바꿔줍니다. 이때 ','문자가없더라도 크기가 하나인 str로 나눠지기 때문에 영향이 없습니다.
      try:
        temp.append(int(''.join(i[2:].split((',')))))
      except:
        temp.append(1) #없는자료 drop을 위한 장치
    globals()['{}_wage_{}'.format(P,Y)]['Sell_value'] = temp

for P in position:
  for Y in year:
    globals()['{}_wage_{}'.format(P,Y)] = globals()['{}_wage_{}'.format(P,Y)][globals()['{}_wage_{}'.format(P,Y)]['Sell_value'] != 1]  # 빼기

In [ ]:
[ ]for P in position:
  for Y in year:\
    globals()['{}_wage_{}'.format(P,Y)].info() #int형으로 바뀐모습

In [ ]:
for P in position:
  for Y in year:
    globals()['{}_{}'.format(P,Y)] = pd.read_csv('{}_{}.csv'.format(P,Y),encoding='utf-8-sig')

In [ ]:
for P in position:
  for Y in year:
    globals()['{}_{}'.format(P,Y)].drop(['Unnamed: 0','Potential','Rating','Wage','Sell_value','Value'],axis = 1, inplace = True, errors = 'ignore')

In [ ]:
for P in position:
  for Y in year:
    print(P,Y,globals()['{}_{}'.format(P,Y)].columns)

In [ ]:
for P in position:
  for Y in year:
    globals()['{}_{}'.format(P,Y)] = pd.merge(globals()['{}_{}'.format(P,Y)],globals()['{}_wage_{}'.format(P,Y)],how = 'inner',on = 'Player')

In [ ]:
for P in position:
  globals()['{}'.format(P)] = pd.concat([globals()['{}_{}'.format(P,'20')],globals()['{}_{}'.format(P,'21')]])
  globals()['{}'.format(P)] = pd.concat([globals()['{}'.format(P)],globals()['{}_{}'.format(P,'22')]])

In [ ]:
#중복 행 제거
for P in position:
  globals()['{}'.format(P)].drop_duplicates()

In [ ]:
# 특정 열의 모든 값이 0인 열 제거

for P in position:
  for F in features:
    if all(globals()['{}'.format(P)][F] == 0):
      globals()['{}'.format(P)] = globals()['{}'.format(P)].drop(F, axis=1)

In [ ]:
Goalkeeper.to_csv('Goalkeeper.csv',encoding ='utf-8-sig')
Forward.to_csv('Forward.csv',encoding ='utf-8-sig')
Midfielder.to_csv('Midfielder.csv',encoding ='utf-8-sig')
Defender.to_csv('Defender.csv',encoding ='utf-8-sig')

OpenMl

In [ ]:
#데이터 불러오기
import pandas as pd
import numpy as np
#데이터 불러오기
#포지션 공격수,수비수,미드필더,골키퍼
position = ['Forward','Midfielder','Defender','Goalkeeper']
#각 데이터를 불러모아 그 csv파일의 이름으로 데이터프레임으로 만들어 주었습니다.
for P in position:
  globals()['{}'.format(P)] = pd.read_csv('{}.csv'.format(P),encoding='utf-8-sig')

In [ ]:
!pip install autoviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.3/356.3 kB 26.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 12.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-2.5.1-py2.py3-none-any.whl size=351210 sha256=de607b78e4244a2d7105cb336fb604931a912165e047059d7bd1c1c21093b541
  Stored in directory: /root/.cache/pip/wheels/51/92/44/e2ef13f803aa

In [ ]:
from autoviz.AutoViz_Class import AutoViz_Class
import matplotlib.pyplot as plt
%matplotlib inline

#plt.style.use('')  #스타일 조정가능

for pos in position:
  globals()['AV_{}'.format(pos)] = AutoViz_Class()
  globals()['AV_{}'.format(pos)].AutoViz(
      filename='',
      dfte =  globals()['{}'.format(pos)],
      depVar = 'value',
      verbose = 0,
      max_rows_analyzed = globals()['{}'.format(pos)].shape[0],
      max_cols_analyzed = globals()['{}'.format(pos)].shape[1]
  )

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install lazypredict

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
Goalkeeper.iloc[:,4:].head()

,Goals,Assists,Appearances,Minutes_Played,Yellow_Cards,Red_Cards,Substituted_On,Substituted_Off,Shots,Shots_On_Target,Goals_From_Header,Touches,Passes,Through_Balls,Interceptions,Tackles,Last_Man_Tackles,Clearances,Headed_Clearances,Aerial_Battles_Won,Own_Goals,Errors_Leading_To_Goal,Penalties_Conceded,Aerial_Battles_Lost,Clean_Sheets,Goals_Conceded,Saves,Penalties_Saved,Punches,High_Claims,Sweeper_Clearances,Throw_Outs,Goal_Kicks,Potential,Rating,age,Wage,Sell_value
0,1.0,0.0,33,2970,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1321,988,0.0,2.0,3.0,1.0,43.0,4.0,14.0,0.0,3.0,1.0,1.0,10.0,32.0,84.0,1.0,17.0,20.0,25.0,165.0,173,90.0,87,27,130000,59000000
1,0.0,1.0,37,3330,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1531,1108,0.0,2.0,0.0,0.0,38.0,3.0,7.0,0.0,0.0,1.0,1.0,6.0,74.0,166.0,1.0,23.0,20.0,14.0,182.0,296,70.0,64,27,33000,8000000
2,0.0,1.0,37,3330,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1531,1108,0.0,2.0,0.0,0.0,38.0,3.0,7.0,0.0,0.0,1.0,1.0,6.0,74.0,166.0,1.0,23.0,20.0,14.0,182.0,296,70.0,64,27,33000,8000000
3,0.0,1.0,36,3240,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1237,959,0.0,0.0,3.0,2.0,17.0,1.0,7.0,0.0,1.0,0.0,1.0,19.0,28.0,66.0,1.0,6.0,19.0,16.0,161.0,153,90.0,84,26,110000,55000000
4,0.0,1.0,36,3240,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1237,959,0.0,0.0,3.0,2.0,17.0,1.0,7.0,0.0,1.0,0.0,1.0,19.0,28.0,66.0,1.0,6.0,19.0,16.0,161.0,153,90.0,84,26,110000,55000000


In [ ]:
from lazypredict.Supervised import LazyRegressor
from sklearn.model_selection import train_test_split

for pos in position:
  globals()['X_{}_train'.format(pos)],globals()['X_{}_test'.format(pos)],globals()['y_{}_train'.format(pos)],globals()['y_{}_test'.format(pos)] =  \
   train_test_split(globals()['{}'.format(pos)].iloc[:,4:-1],globals()['{}'.format(pos)].iloc[:,-1],test_size=0.2,random_state=100)

In [ ]:
print(X_Forward_train.shape)
print(X_Forward_test.shape)
print(y_Forward_train.shape)
print(y_Forward_test.shape)

(259, 37)
(65, 37)
(259,)
(65,)


In [ ]:
clf = LazyRegressor(verbose=0, predictions=True)

In [ ]:
models,predictions = clf.fit(X_Forward_train,X_Forward_test,y_Forward_train,y_Forward_test)
models

100%|██████████| 42/42 [00:06<00:00,  6.67it/s]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
RandomForestRegressor,-0.05,0.56,25027398.73,0.45
BaggingRegressor,-0.07,0.55,25241580.41,0.05
ExtraTreesRegressor,-0.11,0.53,25722748.09,0.28
HistGradientBoostingRegressor,-0.13,0.52,25931496.83,0.18
GradientBoostingRegressor,-0.19,0.50,26624664.01,0.23
LGBMRegressor,-0.22,0.49,26875273.23,0.09
LassoCV,-0.30,0.45,27745581.91,0.25
LassoLarsCV,-0.30,0.45,27749246.23,0.11
LarsCV,-0.30,0.45,27758869.49,0.10


In [ ]:
models,predictions = clf.fit(X_Goalkeeper_train,X_Goalkeeper_test,y_Goalkeeper_train,y_Goalkeeper_test)
models

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 42/42 [00:02<00:00, 17.10it/s]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
Lars,1996129591506123378927861760.00,-550656439036171980818612224.00,378156753331865583616.00,0.05
LassoLars,16.74,-3.34,33578965.73,0.02
KernelRidge,13.93,-2.57,30433775.52,0.03
LinearRegression,9.96,-1.47,25329686.46,0.01
TransformedTargetRegressor,9.96,-1.47,25329686.46,0.01
Lasso,9.53,-1.35,24715224.92,0.02
LinearSVR,7.84,-0.89,22128914.87,0.01
MLPRegressor,7.84,-0.89,22128906.24,0.20
PassiveAggressiveRegressor,7.79,-0.87,22048976.00,0.05


In [ ]:
models,predictions = clf.fit(X_Midfielder_train,X_Midfielder_test,y_Midfielder_train,y_Midfielder_test)
models

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 42/42 [00:06<00:00,  6.83it/s]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
GammaRegressor,0.37,0.68,18462191.79,0.13
OrthogonalMatchingPursuit,0.17,0.59,21177607.75,0.01
LarsCV,0.17,0.58,21234759.68,0.09
LassoLarsIC,0.09,0.55,22157134.56,0.04
RandomForestRegressor,0.09,0.55,22167746.60,0.49
PoissonRegressor,0.09,0.54,22229057.24,0.39
OrthogonalMatchingPursuitCV,0.06,0.53,22523455.93,0.02
ElasticNet,0.06,0.53,22606549.40,0.04
LassoLarsCV,0.03,0.51,22911476.74,0.08


In [ ]:
models,predictions = clf.fit(X_Defender_train,X_Defender_test,y_Defender_train,y_Defender_test)
models

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████| 42/42 [00:11<00:00,  3.77it/s]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
XGBRegressor,0.47,0.66,16458706.51,0.24
OrthogonalMatchingPursuit,0.31,0.56,18830550.53,0.01
PoissonRegressor,0.29,0.54,19104913.30,0.29
LGBMRegressor,0.28,0.54,19221504.42,0.07
ExtraTreesRegressor,0.28,0.54,19271481.75,0.39
LarsCV,0.25,0.52,19593378.65,0.06
LassoLarsCV,0.25,0.52,19593378.65,0.11
LassoCV,0.25,0.52,19593605.34,0.27
GradientBoostingRegressor,0.25,0.52,19641483.11,0.28


 제일 성능이 좋은 `LGMBClassifier` 로 선택


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler #성능 UP
from sklearn.model_selection import train_test_split
# StandardScaler( )로 평균이 0, 분산 1로 데이터 분포도 변환
scaler = StandardScaler()


In [ ]:
for pos in position:
  data_scaled = scaler.fit_transform(globals()['{}'.format(pos)].iloc[:,4:-1])
  target = globals()['{}'.format(pos)].iloc[:,-1]
  x_train, x_test, y_train, y_test = train_test_split(data_scaled, target, test_size=0.3, random_state=46)
  ##########모델 생성
  model = RandomForestRegressor()
  ##########모델 학습
  model.fit(x_train, y_train)
  ##########모델 성능
  print(pos)
  print(model.score(x_test, y_test))
  neg_mse_scores = cross_val_score(model, data_scaled, target, scoring="neg_mean_squared_error", cv = 5)
  rmse_scores  = np.sqrt(-1 * neg_mse_scores)
  avg_rmse = np.mean(rmse_scores)

  # cross_val_score(scoring="neg_mean_squared_error")로 반환된 값은 모두 음수
  print('5 folds 의 평균 RMSE : {0:.3f} '.format(avg_rmse))
  print()